In [21]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch

# Load and preprocess data
def load_data(real_images_dir, sketches_dir, batch_size, image_size):
    # Create image data generators for real-time face images and sketches
    data_generator = ImageDataGenerator(rescale=1./255)
    real_images_generator = data_generator.flow_from_directory(
        real_images_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode=None,
        seed=42
    )
    sketches_generator = data_generator.flow_from_directory(
        sketches_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode=None,
        seed=42
    )
    return real_images_generator, sketches_generator

real_images_data = preprocess_data(real_images_data)
sketches_data = preprocess_data(sketches_data)

# Preprocessing function (same as before)
def preprocess_images(images):
    return (images.astype(np.float32) - 127.5) / 127.5

# Define the GAN model (same as before)
def build_gan(generator, discriminator, learning_rate):
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=learning_rate, beta_1=0.5), metrics=['accuracy'])
    discriminator.trainable = False
    gan_input = layers.Input(shape=(100,))
    generated_image = generator(gan_input)
    gan_output = discriminator(generated_image)
    gan = models.Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(lr=learning_rate, beta_1=0.5))
    return gan

# Define training function (same as before)
def train_gan(generator, discriminator, gan, real_images_generator, sketches_generator, epochs, batch_size):
    steps_per_epoch = min(real_images_generator.n // batch_size, sketches_generator.n // batch_size)
    for epoch in range(epochs):
        real_images_iterator = iter(real_images_generator)
        sketches_iterator = iter(sketches_generator)
        for step in range(steps_per_epoch):
            real_images_batch = preprocess_images(next(real_images_iterator))
            sketches_batch = preprocess_images(next(sketches_iterator))

            # Generate random noise for the generator
            noise = np.random.normal(0, 1, size=[batch_size, 100])
            generated_images = generator.predict(noise)

            # Concatenate real images and generated images
            X = np.concatenate([real_images_batch, sketches_batch])
            # Labels for real and generated images
            y_dis = np.zeros(2 * batch_size)
            y_dis[:batch_size] = 0.9  # Label smoothing

            # Train the discriminator
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_dis)

            # Train the generator
            noise = np.random.normal(0, 1, size=[batch_size, 100])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, y_gen)

        # Print progress
        print(f'Epoch {epoch}/{epochs}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}')

# Define the hyperparameter search space
def build_model(hp):
    # Define hyperparameters to tune
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    hp_batch_size = hp.Choice('batch_size', values=[16, 32, 64])

    # Build the GAN model with tunable hyperparameters
    gan = build_gan(generator, discriminator, learning_rate=hp_learning_rate)
    
    return gan

if __name__ == "__main__":
    # Set random seed for reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)

    # Load data
    real_images_dir = 'path/to/real/images'
    sketches_dir = 'path/to/sketches'
    batch_size = 32
    image_size = (64, 64)  # Adjust image size as needed
    real_images_generator, sketches_generator = load_data(real_images_dir, sketches_dir, batch_size, image_size)

    # Define input shape
    input_shape = image_size + (3,)  # Assuming RGB images

    # Build generator and discriminator (you may adjust these architectures)
    generator = models.Sequential([
        layers.Dense(256, input_shape=(100,)),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Dense(1024),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Dense(np.prod(input_shape), activation='tanh'),
        layers.Reshape(input_shape)
    ])

    discriminator = models.Sequential([
        layers.Flatten(input_shape=input_shape),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(1, activation='sigmoid')
    ])

    # Define the search strategy and start the hyperparameter search
    tuner = RandomSearch(
        build_model,
        objective='val_loss',
        max_trials=5,  # Adjust as needed
        executions_per_trial=1,
        directory='my_dir',
        project_name='gan_hyperparameter_tuning')

    tuner.search_space_summary()  # Optional: Print the search space summary

    # Start the hyperparameter search using the previously defined data generators
    tuner.search(real_images_data, validation_data=(sketches_data, sketches_data), epochs=100)

    # Retrieve the best hyperparameters found during the search
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"Best hyperparameters: {best_hps}")

    # Retrieve and save the best model
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.save('best_generator_model.h5')


NameError: name 'preprocess_data' is not defined

In [19]:
pip install keras-tuner


  Obtaining dependency information for keras-tuner from https://files.pythonhosted.org/packages/2b/39/21f819fcda657c37519cf817ca1cd03a8a025262aad360876d2a971d38b3/keras_tuner-1.4.6-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/128.9 kB ? eta -:--:--
   ------------ -------------------------- 41.0/128.9 kB 991.0 kB/s eta 0:00:01
   ------------------- -------------------- 61.4/128.9 kB 1.1 MB/s eta 0:00:01
   --------------------------------- ---- 112.6/128.9 kB 819.2 kB/s eta 0:00:01
   -------------------------------------- 128.9/128.9 kB 847.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch
from PIL import Image
import os
import os
import cv2
import numpy as np

def preprocess_data(images_data):
    # Flatten the images to a vector of length 100
    flattened_images = images_data.reshape(images_data.shape[0], -1)  # Reshape to (None, 100)
    # Normalize the pixel values to the range [0, 1]
    normalized_images = flattened_images.astype('float32') / 255.0
    return normalized_images


def convert_to_jpg(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            continue  # Skip files already in .jpg format
        try:
            img = Image.open(os.path.join(directory, filename))
            new_filename = os.path.splitext(filename)[0] + ".jpg"
            img.save(os.path.join(directory, new_filename))
            os.remove(os.path.join(directory, filename))
        except Exception as e:
            print(f"Failed to convert {filename}: {e}")
# Load and preprocess data


def load_data(real_images_dir, sketches_dir, batch_size, image_size):
    # Function to load and preprocess images
    def preprocess_images(image_paths, target_size):
        images = []
        for path in image_paths:
            img = cv2.imread(path)
            img = cv2.resize(img, target_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = img.astype(np.float32) / 255.0        # Normalize pixel values
        # Flatten the image to a vector of length 100
            img = img.flatten()
            images.append(img)
        return np.array(images)



    # Load real images
    real_images_paths = [os.path.join(real_images_dir, filename) for filename in os.listdir(real_images_dir)]
    real_images = preprocess_images(real_images_paths, image_size)

    # Load sketches
    sketches_paths = [os.path.join(sketches_dir, filename) for filename in os.listdir(sketches_dir)]
    sketches = preprocess_images(sketches_paths, image_size)

    print("Number of real images: ", len(real_images))
    print("Number of sketches: ", len(sketches))

    return real_images, sketches





# Preprocessing function
def preprocess_images(images):
    return (images.astype(np.float32) - 127.5) / 127.5

# Define the GAN model
def build_gan(hp):
    # Define hyperparameters to tune
    hp_activation = hp.Choice('activation', values=['relu', 'leaky_relu'])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    hp_batch_size = hp.Choice('batch_size', values=[16, 32, 64])
    hp_units = hp.Choice('units', values=[128, 256, 512])
    hp_hidden_layers = hp.Int('hidden_layers', min_value=1, max_value=3)

    # Build the generator
    generator = models.Sequential()
    generator.add(layers.Dense(hp_units, input_shape=(100,), activation=hp_activation))
    generator.add(layers.BatchNormalization())
    for _ in range(hp_hidden_layers - 1):
        generator.add(layers.Dense(hp_units, activation=hp_activation))
        generator.add(layers.BatchNormalization())
    generator.add(layers.Dense(100, activation='tanh'))  # Output shape should be (100,)

    generator.add(layers.Reshape((64, 64, 3)))  # Adjust dimensions as needed


    # Build the discriminator
    discriminator = models.Sequential()
    discriminator.add(layers.Flatten(input_shape=input_shape))
    discriminator.add(layers.Dense(hp_units, activation=hp_activation))
    for _ in range(hp_hidden_layers - 1):
        discriminator.add(layers.Dense(hp_units, activation=hp_activation))
    discriminator.add(layers.Dense(1, activation='sigmoid'))

    # Compile discriminator
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=hp_learning_rate, beta_1=0.5), metrics=['accuracy'])

    # Compile GAN
    gan_input = layers.Input(shape=(100,))
    generated_image = generator(gan_input)
    gan_output = discriminator(generated_image)
    gan = models.Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=hp_learning_rate, beta_1=0.5))

    return gan

# Define training function
def train_gan(generator, discriminator, gan, real_images_generator, sketches_generator, epochs, batch_size):
    steps_per_epoch = min(real_images_generator.n // batch_size, sketches_generator.n // batch_size)
    for epoch in range(epochs):
        real_images_iterator = iter(real_images_generator)
        sketches_iterator = iter(sketches_generator)
        for step in range(steps_per_epoch):
            real_images_batch = preprocess_images(next(real_images_iterator))
            sketches_batch = preprocess_images(next(sketches_iterator))

            # Generate random noise for the generator
            noise = np.random.normal(0, 1, size=[batch_size, 100])
            generated_images = generator.predict(noise)

            # Concatenate real images and generated images
            X = np.concatenate([real_images_batch, sketches_batch])
            # Labels for real and generated images
            y_dis = np.zeros(2 * batch_size)
            y_dis[:batch_size] = 0.9  # Label smoothing

            # Train the discriminator
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_dis)

            # Train the generator
            noise = np.random.normal(0, 1, size=[batch_size, 100])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, y_gen)

        # Print progress
        print(f'Epoch {epoch}/{epochs}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}')

# Main function
# Load data
real_images_dir = r"C:\Users\paivi\Downloads\Telegram Desktop\Real"
sketches_dir = r"C:\Users\paivi\Downloads\Telegram Desktop\Sketch_1"

batch_size = 32
image_size = (64, 64)  # Adjust image size as needed
real_images_data, sketches_data = load_data(real_images_dir, sketches_dir, batch_size, image_size)

# Preprocess the loaded data
# Preprocess the loaded data
real_images_data = preprocess_data(real_images_data)
sketches_data = preprocess_data(sketches_data)


# Define input shape
input_shape = image_size + (3,)  # Assuming RGB images

# Define the search strategy and start the hyperparameter search
tuner = RandomSearch(
    build_gan,
    objective='val_loss',
    max_trials=5,  # Adjust as needed
    executions_per_trial=5,
    directory='my_dir',
    project_name='gan_hyperparameter_tuning')

tuner.search_space_summary()  # Optional: Print the search space summary

# Start the hyperparameter search using the preprocessed data
tuner.search(real_images_data, validation_data=(sketches_data, sketches_data), epochs=100)

# Retrieve the best hyperparameters found during the search
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps}")

# Retrieve and save the best model
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save('best_generator_model.h5')


Number of real images:  1000
Number of sketches:  1000
Reloading Tuner from my_dir\gan_hyperparameter_tuning\tuner0.json
Search space summary
Default search space size: 5
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'leaky_relu'], 'ordered': False}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 1e-05], 'ordered': True}
batch_size (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}
units (Choice)
{'default': 128, 'conditions': [], 'values': [128, 256, 512], 'ordered': True}
hidden_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
leaky_relu        |relu              |activation
1e-05             |0.001             |learning_rate
64                |32                |batch_size
256               |512               |units
3                 |1    

Traceback (most recent call last):
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
            ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\tuner.py", line

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
            ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pa

# MAIN

# --------------------------------------------------------------------------------------------------------

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch
from PIL import Image
import os
import cv2

# Function to preprocess data
def preprocess_data(images_data):
    # Flatten the images to a vector of length 100
    flattened_images = images_data.reshape(images_data.shape[0], -1)  # Reshape to (None, 100)
    # Normalize the pixel values to the range [0, 1]
    normalized_images = flattened_images.astype('float32') / 255.0
    return normalized_images

# Function to convert images to JPG format
def convert_to_jpg(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            continue  # Skip files already in .jpg format
        try:
            img = Image.open(os.path.join(directory, filename))
            new_filename = os.path.splitext(filename)[0] + ".jpg"
            img.save(os.path.join(directory, new_filename))
            os.remove(os.path.join(directory, filename))
        except Exception as e:
            print(f"Failed to convert {filename}: {e}")

# Function to load and preprocess images
def load_data(real_images_dir, sketches_dir, batch_size, image_size):
    # Function to preprocess images
    def preprocess_images(image_paths, target_size):
        images = []
        for path in image_paths:
            img = cv2.imread(path)
            img = cv2.resize(img, target_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = img.astype(np.float32) / 255.0        # Normalize pixel values
            images.append(img)
        return np.array(images)

    # Load real images
    real_images_paths = [os.path.join(real_images_dir, filename) for filename in os.listdir(real_images_dir)]
    real_images = preprocess_images(real_images_paths, image_size)

    # Load sketches
    sketches_paths = [os.path.join(sketches_dir, filename) for filename in os.listdir(sketches_dir)]
    sketches = preprocess_images(sketches_paths, image_size)

    print("Number of real images:", len(real_images))
    print("Number of sketches:", len(sketches))

    return real_images, sketches

# Define input shape
input_shape = (64, 64, 3)

# Define the GAN model
# Define the GAN model
def build_gan(hp):
    # Define hyperparameters to tune
    hp_activation = hp.Choice('activation', values=['relu', 'leaky_relu'])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    hp_batch_size = hp.Choice('batch_size', values=[16, 32, 64])
    hp_units = hp.Choice('units', values=[128, 256, 512])
    hp_hidden_layers = hp.Int('hidden_layers', min_value=1, max_value=3)

    # Build the generator
    generator = models.Sequential()
    generator.add(layers.Dense(hp_units, input_shape=(100,), activation=hp_activation))
    generator.add(layers.BatchNormalization())
    for _ in range(hp_hidden_layers - 1):
        generator.add(layers.Dense(hp_units, activation=hp_activation))
        generator.add(layers.BatchNormalization())
    generator.add(layers.Dense(100, activation='tanh'))  # Output shape should be (100,)

    generator.add(layers.Reshape((10, 10, 1)))  # Adjust dimensions as needed

    # Build the discriminator
    discriminator = models.Sequential()
    discriminator.add(layers.Flatten(input_shape=(10, 10, 1)))  # Adjust input shape accordingly
    discriminator.add(layers.Dense(hp_units, activation=hp_activation))
    for _ in range(hp_hidden_layers - 1):
        discriminator.add(layers.Dense(hp_units, activation=hp_activation))
    discriminator.add(layers.Dense(1, activation='sigmoid'))

    # Compile discriminator
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=hp_learning_rate, beta_1=0.5), metrics=['accuracy'])

    # Compile GAN
    gan_input = layers.Input(shape=(100,))
    generated_image = generator(gan_input)
    gan_output = discriminator(generated_image)
    gan = models.Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=hp_learning_rate, beta_1=0.5))

    return gan


# Define training function
def train_gan(generator, discriminator, gan, real_images, sketches, epochs, batch_size):
    real_images_generator = ImageDataGenerator().flow(real_images, batch_size=batch_size, shuffle=True)
    sketches_generator = ImageDataGenerator().flow(sketches, batch_size=batch_size, shuffle=True)
    steps_per_epoch = min(len(real_images) // batch_size, len(sketches) // batch_size)
    for epoch in range(epochs):
        for step in range(steps_per_epoch):
            real_images_batch = next(real_images_generator)
            sketches_batch = next(sketches_generator)

            # Generate random noise for the generator
            noise = np.random.normal(0, 1, size=[batch_size, 100])
            generated_images = generator.predict(noise)

            # Concatenate real images and generated images
            X = np.concatenate([real_images_batch, sketches_batch])
            # Labels for real and generated images
            y_dis = np.zeros(2 * batch_size)
            y_dis[:batch_size] = 0.9  # Label smoothing

            # Train the discriminator
            discriminator.trainable = True
            d_loss = discriminator.train_on_batch(X, y_dis)

            # Train the generator
            noise = np.random.normal(0, 1, size=[batch_size, 100])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, y_gen)

        # Print progress
        print(f'Epoch {epoch}/{epochs}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}')

# Main function
if __name__ == "__main__":
    # Load data
    real_images_dir = r"C:\Users\paivi\Downloads\Telegram Desktop\Real"
    sketches_dir = r"C:\Users\paivi\Downloads\Telegram Desktop\Sketch_1"
    batch_size = 32
    image_size = (64, 64)  # Adjust image size as needed
    real_images, sketches = load_data(real_images_dir, sketches_dir, batch_size, image_size)

    # Preprocess the loaded data
    real_images_data = preprocess_data(real_images)
    sketches_data = preprocess_data(sketches)

    # Define the search strategy and start the hyperparameter search
    tuner = RandomSearch(
        build_gan,
        objective='val_loss',
        max_trials=5,
        executions_per_trial=1,
        directory='my_dir',
        project_name='gan_hyperparameter_tuning',
        hyperparameters={
        'activation': hp.Choice('activation', values=['leaky_relu', 'relu']),
        'learning_rate': hp.Choice('learning_rate', values=[1e-06, 1e-05]),
        'batch_size': hp.Choice('batch_size', values=[128, 256]),
        'units': hp.Choice('units', values=[512, 1024]),
        'hidden_layers': hp.Choice('hidden_layers', values=[2, 3])
    }
)
    tuner.search(real_images_data, validation_data=(sketches_data, sketches_data), epochs=100)

# Retrieve the best hyperparameters found during the search
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"Best hyperparameters: {best_hps}")

# Retrieve and save the best model
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.save('best_generator_model.h5')

Number of real images: 1000
Number of sketches: 1000


NameError: name 'hp' is not defined

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import RandomSearch
from kerastuner import HyperParameters  # Import HyperParameters module
from PIL import Image
import os
import cv2

# Function to preprocess data
# Function to preprocess data
def preprocess_data(images_data, target_size):
    # Resize images to target size and flatten
    resized_images = [cv2.resize(img, target_size).flatten() for img in images_data]
    # Normalize pixel values to [0, 1]
    normalized_images = np.array(resized_images) / 255.0
    return normalized_images

# Function to load and preprocess images
def load_data(real_images_dir, sketches_dir, batch_size, image_size):
    # Load real images
    real_images_paths = [os.path.join(real_images_dir, filename) for filename in os.listdir(real_images_dir)]
    real_images = [cv2.imread(path) for path in real_images_paths]

    # Load sketches
    sketches_paths = [os.path.join(sketches_dir, filename) for filename in os.listdir(sketches_dir)]
    sketches = [cv2.imread(path) for path in sketches_paths]

    print("Number of real images:", len(real_images))
    print("Number of sketches:", len(sketches))

    return real_images, sketches

# Define the GAN model
def build_gan(hp):
    # Define hyperparameters to tune
    hp_activation = hp.Choice('activation', values=['relu', 'leaky_relu'])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    hp_batch_size = hp.Choice('batch_size', values=[16, 32, 64])
    hp_units = hp.Choice('units', values=[128, 256, 512])
    hp_hidden_layers = hp.Int('hidden_layers', min_value=1, max_value=3)

    # Build the generator
    generator = models.Sequential()
    generator.add(layers.Dense(hp_units, input_shape=(100,), activation=hp_activation))
    generator.add(layers.BatchNormalization())
    for _ in range(hp_hidden_layers - 1):
        generator.add(layers.Dense(hp_units, activation=hp_activation))
        generator.add(layers.BatchNormalization())
    generator.add(layers.Dense(64*64*3, activation='sigmoid'))  # Output shape should match the flattened image size

    # Reshape the output to match image dimensions
    generator.add(layers.Reshape((64, 64, 3)))  # Adjust dimensions as needed

    # Build the discriminator
    discriminator = models.Sequential()
    discriminator.add(layers.Flatten(input_shape=(64, 64, 3)))  # Adjust input shape accordingly
    discriminator.add(layers.Dense(hp_units, activation=hp_activation))
    for _ in range(hp_hidden_layers - 1):
        discriminator.add(layers.Dense(hp_units, activation=hp_activation))
    discriminator.add(layers.Dense(1, activation='sigmoid'))

    # Compile discriminator
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=hp_learning_rate, beta_1=0.5), metrics=['accuracy'])

    # Compile GAN
    gan_input = layers.Input(shape=(100,))
    generated_image = generator(gan_input)
    gan_output = discriminator(generated_image)
    gan = models.Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=hp_learning_rate, beta_1=0.5))

    return gan

# Main function
if __name__ == "__main__":
    # Load data
    real_images_dir = r"C:\Users\paivi\Downloads\Telegram Desktop\Real"
    sketches_dir = r"C:\Users\paivi\Downloads\Telegram Desktop\Sketch_1"
    batch_size = 32
    image_size = (64, 64)  # Adjust image size as needed
    real_images, sketches = load_data(real_images_dir, sketches_dir, batch_size, image_size)

    # Preprocess the loaded data
    real_images_data = preprocess_data(real_images, target_size=(64, 64))
    sketches_data = preprocess_data(sketches, target_size=(64, 64))

    # Define the search strategy and start the hyperparameter search
    tuner = RandomSearch(
        build_gan,
        objective='val_loss',
        max_trials=5,
        executions_per_trial=1,
        directory='my_dir',
        project_name='gan_hyperparameter_tuning',
        overwrite=True  # Overwrite existing results
    )
    tuner.search(real_images_data, validation_data=(sketches_data, sketches_data), epochs=100)

    # Retrieve the best hyperparameters found during the search
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"Best hyperparameters: {best_hps}")

    # Retrieve and save the best model
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.save('best_generator_model.h5')



Trial 2 Complete [00h 00m 01s]

Best val_loss So Far: None
Total elapsed time: 00h 00m 03s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
leaky_relu        |leaky_relu        |activation
0.0001            |0.0001            |learning_rate
32                |32                |batch_size
128               |256               |units
3                 |3                 |hidden_layers

Epoch 1/100


Traceback (most recent call last):
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\paivi\anacon

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\paivi\anaconda3\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 233, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
              ^^^^^^^